<center>

<h1 style="text-align:center"> Modules and Functional Data Structures </h1>
<h2 style="text-align:center"> CSCI7000-11 S23: Principles of Functional Programming </h2>
</center>

## Review

* Previously
  + How to build small programs
  
* This lecture
  + How to build at scale: Structures, Signatures, Functors.
  + Functional Data Structures
  
$
\require{color}
\newcommand{\cred}[1]{{\color{\red}{\text{#1}}}}
\newcommand{\cgreen}[1]{{\color{\green}{\text{#1}}}}
$

In [2]:
type 'a queue = 'a list

type 'a queue = 'a list


In [3]:
let rec push (x:'a) (q: 'a queue) : 'a queue = q@[x] 

val push : 'a -> 'a queue -> 'a queue = <fun>


In [4]:
let rec pop (q: 'a queue) : ('a*'a queue) = match q with
  | [] -> raise (Invalid_argument "queue is empty!")
  | h::t -> (h,t)

val pop : 'a queue -> 'a * 'a queue = <fun>


In [5]:
type 'a queue = 
 | QNil
 | QCons of 'a queue * 'a

type 'a queue = QNil | QCons of 'a queue * 'a


In [6]:
(*
 Idea: implement a queue as a pair of lists
  * when q = (f,r), the real queue is f @ (List.rev r)
  * E.g: when q = ([3;9;1], [5;2;10]), the actual queue is [3;9;1;10;2;5]
*)

type 'a queue = 'a list * 'a list

type 'a queue = 'a list * 'a list


In [7]:
let push (q:'a queue) (x:'a) : 'a queue = match q with
 | ([],[]) -> ([x],[])
 | (f,r) -> (f, x::r)

val push : 'a queue -> 'a -> 'a queue = <fun>


In [8]:
let pop (q: 'a queue) : ('a*'a queue) = match q with
  | ([],[]) -> raise (Invalid_argument "empty queue!")
  | ([x],r) ->
      let popped_el = x in
      let f' = List.rev r in
      let r' = [] in
      (popped_el, (f',r'))
  | ([], r) -> failwith "This case cannot happen!"
  | (h::t,r) -> (h, (t,r))

val pop : 'a queue -> 'a * 'a queue = <fun>


In [9]:
let empty_queue () : 'a queue = ([],[])

val empty_queue : unit -> 'a queue = <fun>


In [10]:
let q1 = empty_queue () in
let q2 = push q1 3 in
let q3 = push q2 4 in
let (el,q4) = pop q3 in
(el, q4)

- : int * int queue = (3, ([4], []))


In [11]:
(*
  Abstract interface (signature) for queue data structure.
  Captures our desiderata for the data structure.
*)
module type QUEUE = sig
  type 'a t
  val empty : unit -> 'a t
  val pop: 'a t -> ('a * 'a t) 
  val push: 'a t -> 'a -> 'a t
  val to_string: 'a t -> string
end

module type QUEUE =
  sig
    type 'a t
    val empty : unit -> 'a t
    val pop : 'a t -> 'a * 'a t
    val push : 'a t -> 'a -> 'a t
    val to_string : 'a t -> string
  end


In [12]:
module Queue: QUEUE = struct
  type 'a t = 'a list * 'a list
  
  let empty () = ([],[])
  
  let push (q:'a queue) (x:'a) : 'a queue = match q with
   | ([],[]) -> ([x],[])
   | (f,r) -> (f, x::r)
 
 let pop (q: 'a queue) : ('a*'a queue) = match q with
  | ([],[]) -> raise (Invalid_argument "empty queue!")
  | ([x],r) ->
      let popped_el = x in
      let f' = List.rev r in
      let r' = [] in
      (popped_el, (f',r'))
  | ([], r) -> failwith "This case cannot happen!"
  | (h::t,r) -> (h, (t,r))

 let to_string q = failwith "Not implemented!"
 
end

module Queue : QUEUE


In [13]:

let (q1: int Queue.t) = Queue.empty ()


val q1 : int Queue.t = <abstr>


In [14]:
let (q2: int Queue.t) = Queue.push q1 3

val q2 : int Queue.t = <abstr>


In [15]:
match q2 with 
  | (f,r) -> f

error: compile_error

In [20]:
module type SET = sig
  type 'a t
  val empty: unit -> 'a t
  val insert: 'a t -> 'a -> 'a t
  val is_empty: 'a t -> bool
  val exists: 'a t -> 'a -> bool
end

module type SET =
  sig
    type 'a t
    val empty : unit -> 'a t
    val insert : 'a t -> 'a -> 'a t
    val is_empty : 'a t -> bool
    val exists : 'a t -> 'a -> bool
  end


In [34]:
module type SET = sig
  type t
  module S:COMPARABLE
  val empty: unit -> t
  val insert: t -> S.t -> t
  val is_empty: t -> bool
  val exists: t -> S.t -> bool
end

module type SET =
  sig
    type t
    module S : COMPARABLE
    val empty : unit -> t
    val insert : t -> S.t -> t
    val is_empty : t -> bool
    val exists : t -> S.t -> bool
  end


In [22]:
module type COMPARABLE = sig
  type t
  val compare: t -> t -> int
end

module type COMPARABLE = sig type t val compare : t -> t -> int end


In [36]:
(*
  We have constrained polymorphism in search tree.
  Instead of trees of containing values of *any* type, we now
  have trees containing values that are of any type that is comparable.
*)

module SearchTree (S: COMPARABLE):SET = struct

  module S = S

  type t = 
    | Leaf
    | Branch of t * S.t * t
    
  let empty () = Leaf
    
  let rec insert t x = match t with
   | Leaf -> Branch(Leaf, x, Leaf)
   | Branch (lt, y, rt) -> 
       if S.compare x y <= 0 then Branch(insert lt x, y, rt)
       else Branch(lt, y, insert rt x)
       
  let is_empty t = match t with
   | Leaf -> true
   | _ -> false
   
  let rec exists t x  = failwith "not impl."
end

module SearchTree : functor (S : COMPARABLE) -> SET


In [19]:
(<=)

- : 'a -> 'a -> bool = <fun>


In [26]:
module ComparableString = struct
  type t = string
  let compare s1 s2 = 
      if s1 < s2 then 0-1 
      else if s1 = s2 then 0 
      else 1
end

module ComparableString :
  sig type t = string val compare : 'a -> 'a -> int end


In [29]:
module TreeOfStrings = SearchTree(String)

module TreeOfStrings :
  sig
    type t = SearchTree(String).t = Leaf | Branch of t * String.t * t
    val empty : unit -> t
    val insert : t -> String.t -> t
    val is_empty : t -> bool
    val exists : 'a -> 'b -> 'c
  end


In [28]:
let e = TreeOfStrings.empty () in
TreeOfStrings.insert e "Hello"

- : TreeOfStrings.t =
TreeOfStrings.Branch (TreeOfStrings.Leaf, "Hello", TreeOfStrings.Leaf)


In [31]:
module TreeOfInts = SearchTree(Int)

module TreeOfInts :
  sig
    type t = SearchTree(Int).t = Leaf | Branch of t * Int.t * t
    val empty : unit -> t
    val insert : t -> Int.t -> t
    val is_empty : t -> bool
    val exists : 'a -> 'b -> 'c
  end


In [37]:
(** 

  Revisiting immutability
*)

let l1 = [1;2;3;4] in
let l2 = 5::l1 in
let l3 = l1 @ l2 in
print l1;
l3

- : int list = [1; 2; 3; 4; 5; 1; 2; 3; 4]


In [38]:
module type T = sig 
  type t 
  val v : t 
end

module type T = sig type t val v : t end


In [45]:
module Id (X : T) : (T with type t = X.t) = X

module Id : functor (X : T) -> sig type t = X.t val v : t end


In [41]:
module M:T = struct 
  type t = int
  let v = 10
end

module M : T


In [48]:
module M1: (T with type t = M.t) = Id(M)

module M1 : sig type t = M.t val v : t end


In [49]:
M1.v = M.v

- : bool = true
